In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

## Data Cleansing & Pre-Processing

In [2]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [3]:
world_gdp = pd.read_csv("data/world_gdp.csv")
trade_amount = pd.read_csv("data/Trade in goods and services forecast.csv")

In [4]:
# date
data = pd.read_csv('../../Data/Jeju/201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))

In [5]:
data.REG_YYMM = pd.to_datetime(data.REG_YYMM, format='%Y%m')

In [6]:
world_gdp['Unnamed: 0'] = pd.to_datetime(world_gdp['Unnamed: 0'])
trade_amount['Unnamed: 0'] = pd.to_datetime(trade_amount['Unnamed: 0'])

In [7]:
data = data.drop(['REG_YYMM'], axis=1)

In [8]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

# 카테고리 별로 묶어서 AMT를 각각 예측
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [9]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1057394 non-null  object
 1   STD_CLSS_NM   1057394 non-null  object
 2   HOM_SIDO_NM   1057394 non-null  object
 3   AGE           1057394 non-null  object
 4   SEX_CTGO_CD   1057394 non-null  int64 
 5   FLC           1057394 non-null  int64 
 6   year          1057394 non-null  int64 
 7   month         1057394 non-null  int64 
 8   CSTMR_CNT     1057394 non-null  int64 
 9   AMT           1057394 non-null  int64 
 10  CNT           1057394 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 88.7+ MB


## Exploratory Data Analysis

## Feature Engineering & Initial Modeling

In [11]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [12]:
train_features['kor_gdp'] = 0
train_features['trade_amount'] = 0

for date, i in zip(world_gdp['Unnamed: 0'], range(len(world_gdp))):
    train_features.loc[(train_features['year'] == date.year) & (train_features['month'] == date.month), 'kor_gdp'] = world_gdp.iloc[i].KOR_gdp
    
for date, i in zip(trade_amount['Unnamed: 0'], range(len(trade_amount))):
    train_features.loc[(train_features['year'] == date.year) & (train_features['month'] == date.month), 'trade_amount'] = trade_amount.iloc[i].Value
    

In [13]:
train_features.trade_amount = np.log1p(train_features['trade_amount'])

## Model Tuning & Evaluation

In [ ]:
# Random Forest
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

In [24]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)

In [25]:
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 
                                        'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month'])

In [26]:
temp['kor_gdp'] = 0
temp['trade_amount'] = 0

for date, i in zip(world_gdp['Unnamed: 0'], range(len(world_gdp))):
    temp.loc[(temp['year'] == date.year) & (temp['month'] == date.month), 'kor_gdp'] = world_gdp.iloc[i].KOR_gdp
    
for date, i in zip(trade_amount['Unnamed: 0'], range(len(trade_amount))):
    temp.loc[(temp['year'] == date.year) & (temp['month'] == date.month), 'trade_amount'] = np.log1p(trade_amount.iloc[i].Value)
    

In [27]:
temp_temp = temp.copy()

In [111]:
# 예측(Random Forest)
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [112]:
# 디코딩 (Random Forest)
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [113]:
# 제출 파일 만들기(Random Forest)
submission = pd.read_csv('../../Data/Jeju/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission/kor_gdp_trade_amount_RF.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,4.749459e+08
1,202004,강원,골프장 운영업,5.737034e+09
2,202004,강원,과실 및 채소 소매업,1.879838e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.036658e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.000212e+08


In [35]:
train_features

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,kor_gdp,trade_amount
149245,2,4,2,2,1,2,2019,4,4.118698,27.141631
554956,8,26,11,1,1,2,2020,2,-5.029080,27.121861
918516,14,33,14,4,2,4,2019,5,4.118698,27.141631
425414,7,18,8,2,2,1,2019,10,5.350648,27.157812
640949,9,39,16,5,2,5,2020,1,-5.029080,27.121861
...,...,...,...,...,...,...,...,...,...,...
359783,6,16,9,1,1,1,2019,12,5.350648,27.157812
152315,2,7,11,3,2,3,2020,3,-5.029080,27.121861
963395,15,19,9,5,1,5,2020,2,-5.029080,27.121861
117952,1,31,4,1,1,2,2019,7,1.510299,27.154597


In [37]:
train_target

149245    12.524530
554956    11.399768
918516    20.495010
425414    14.850157
640949    12.762830
            ...    
359783    15.138871
152315    12.345448
963395    13.691082
117952    11.584697
305711    12.944458
Name: AMT, Length: 1057394, dtype: float64

In [14]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [91]:
k = int(len(train_features)*0.8)

In [92]:
"""x_train = train_features[:k]
y_train = train_target[:k]
x_val = train_features[k:]
y_val = train_target[k:]"""

In [17]:
 x_train, x_val, y_train, y_val = train_test_split(train_features, train_target, random_state=42)

In [18]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [19]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'tweedie',
    'metric': 'rmse',
    'tweedie_variance_power':1.1,
    'max_depth':11,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0}

In [20]:
model = lgb.train(params,
                  train_ds,
                  2000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.98669
[200]	valid_0's rmse: 1.80375
[300]	valid_0's rmse: 1.69015
[400]	valid_0's rmse: 1.61141
[500]	valid_0's rmse: 1.53833
[600]	valid_0's rmse: 1.47842
[700]	valid_0's rmse: 1.42226
[800]	valid_0's rmse: 1.37346
[900]	valid_0's rmse: 1.33202
[1000]	valid_0's rmse: 1.28655
[1100]	valid_0's rmse: 1.25005
[1200]	valid_0's rmse: 1.22082
[1300]	valid_0's rmse: 1.19277
[1400]	valid_0's rmse: 1.16522
[1500]	valid_0's rmse: 1.14432
[1600]	valid_0's rmse: 1.12369
[1700]	valid_0's rmse: 1.10586
[1800]	valid_0's rmse: 1.08761
[1900]	valid_0's rmse: 1.07134
[2000]	valid_0's rmse: 1.05942
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 1.05942


In [28]:
temp = temp_temp

In [29]:
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [30]:
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [31]:

submission = pd.read_csv('../../Data/Jeju/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission/lightgbm_korea_gdp_trade_amount_2.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.915810e+08
1,202004,강원,골프장 운영업,2.495096e+09
2,202004,강원,과실 및 채소 소매업,7.749600e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,4.122657e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.402082e+07
